## import required packages

In [1]:
import os
print(os.getcwd())

os.chdir("/Users/Jennifer/Desktop/CSE291B/actionable-recourse-git-master-repo/examples/paper")

from initialize import *
import itertools
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

/Users/Jennifer/Desktop/CSE291B/actionable-recourse-git-master-repo/examples


## read in the givemecredit dataset: 

In [2]:
data_name = 'givemecredit'
data_file = data_dir / ('%s/cs-training.csv' % (data_name))

output_dir = results_dir / data_name
raw_df = pd.concat([pd.read_csv(data_file, index_col=0), ])

In [3]:
raw_df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,...,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,...,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,...,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,...,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,...,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,...,7,0,1,0,0.0


In [4]:
raw_df.dtypes

SeriousDlqin2yrs                          int64
RevolvingUtilizationOfUnsecuredLines    float64
age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
MonthlyIncome                           float64
NumberOfOpenCreditLinesAndLoans           int64
NumberOfTimes90DaysLate                   int64
NumberRealEstateLoansOrLines              int64
NumberOfTime60-89DaysPastDueNotWorse      int64
NumberOfDependents                      float64
dtype: object

In [5]:
## okay so SINCE the logreg models are not converging maybe we should try on a smaller dataset and see if that helps
# from random import sample

# raw_df = raw_df.sample(frac=0.25, replace=False, random_state=1)



## now set up our actionset

In [6]:
y = 1 - raw_df['SeriousDlqin2yrs'].reset_index(drop=True)
X = raw_df.drop('SeriousDlqin2yrs', axis=1).fillna(0).reset_index(drop=True)

X_audit_holdout = X

# actionset
action_set = ActionSet(X=X_audit_holdout)
action_set['age'].mutable = False
action_set['NumberOfDependents'].mutable = False
action_set['DebtRatio'].step_direction = -1


## separate data into training and test

In [7]:
## split datasets for audit
X_clf, X_audit_holdout, y_clf, y_audit_holdout = train_test_split(X, y, test_size=.25)

In [8]:
print(X_clf.shape)
print(X_audit_holdout.shape)
print(y_clf.shape)
print(y_audit_holdout.shape)

(112500, 10)
(37500, 10)
(112500,)
(37500,)


In [9]:
## maybe we need to rescale our data
scaler = StandardScaler().fit(X_clf)
X_clf_scaled = scaler.transform(X_clf)

In [10]:
X_audit_holdout_scaled = scaler.transform(X_audit_holdout)

## generate flipset

In [12]:
clf = (
    LogisticRegression(C=.02, max_iter=1000, penalty='l1', solver='liblinear', tol=1e-8)
       .fit(X_clf_scaled, y_clf)
)

## if the above code does not work we can try the grid search
## grid search

# from sklearn.model_selection import GridSearchCV

# clf = LogisticRegression(max_iter=1000, solver='lbfgs')

# ## Exhaustive search over specified parameter values for an estimator.
# ## C is a inverse of regularization strength, smaller values == stronger regularization
# ## cv = cross validation generator, optional. default is 5-fold.
# ## scoring https://scikit-learn.org/stable/modules/model_evaluation.html
# ## Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
# grid = GridSearchCV(
#   clf, param_grid={'C': np.logspace(-4, 3)},
#   cv=10,
#   scoring='roc_auc',
#   return_train_score=True
# )
# grid.fit(X_clf, y_clf)
# clf = grid.best_estimator_

In [23]:
### Create Flipset
yhat = clf.predict(X = X_clf_scaled)
coefficients, intercept = undo_coefficient_scaling(clf, scaler = scaler)
action_set.align(coefficients)
predicted_neg = np.flatnonzero(yhat < 1)

U = X_clf.iloc[predicted_neg].values

In [24]:
## 
k = 4
fb = Flipset(x = U[k], action_set = action_set, coefficients = coefficients, intercept = intercept)
fb.populate(enumeration_type = 'distinct_subsets', total_items = 14)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_MIP_Cuts_Covers                         -1
CPXPARAM_MIP_Display                             1
CPXPARAM_MIP_Cuts_MIRCut                         -1
CPXPARAM_MIP_Pool_Capacity                       100
CPXPARAM_MIP_Pool_Replace                        2
CPXPARAM_MIP_Cuts_ZeroHalfCut                    -1
CPXPARAM_MIP_Cuts_LocalImplied                   3
CPXPARAM_MIP_Limits_RepairTries                  20
CPXPARAM_MIP_Tolerances_AbsMIPGap                2.2204460492503131e-16
CPXPARAM_MIP_Tolerances_MIPGap                   2.2204460492503131e-16
CPXPARAM_MIP_Tolerances_Integrality              0
MIP Presolve eliminated 5 redundant SOS constraints.
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
Reduced MI

CPXPARAM_MIP_Tolerances_Integrality              0
1 of 2 MIP starts provided solutions.
MIP start 'm2' defined initial solution with objective 50.8032.
Retaining values of one MIP start for possible repair.
MIP Presolve eliminated 5 redundant SOS constraints.
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
Reduced MIP has 16 rows, 144 columns, and 308 nonzeros.
Reduced MIP has 139 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.61 ticks)
Probing time = 0.00 sec. (0.78 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 16 rows, 144 columns, and 308 nonzeros.
Reduced MIP has 139 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.48 ticks)
Probing time = 0.00 sec. (0.59 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.

Node log . . .
Best integer =   5.080323e+01  Node =       0  Best node =   0.000000e+00
Best 

CPXPARAM_MIP_Tolerances_MIPGap                   2.2204460492503131e-16
CPXPARAM_MIP_Tolerances_Integrality              0
2 of 3 MIP starts provided solutions.
MIP start 'm3' defined initial solution with objective 8.5537.
Retaining values of one MIP start for possible repair.
MIP Presolve eliminated 5 redundant SOS constraints.
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
Reduced MIP has 19 rows, 144 columns, and 323 nonzeros.
Reduced MIP has 139 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.63 ticks)
Probing time = 0.00 sec. (0.80 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 19 rows, 144 columns, and 323 nonzeros.
Reduced MIP has 139 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.49 ticks)
Probing time = 0.00 sec. (0.60 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.

Node log . . .
Best int

Detecting symmetries...
Reduced MIP has 22 rows, 144 columns, and 338 nonzeros.
Reduced MIP has 139 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.50 ticks)
Probing time = 0.00 sec. (0.61 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.

Node log . . .
Best integer =   8.604143e+00  Node =       0  Best node =   0.000000e+00
Repair heuristic found nothing.
Detecting symmetries...

Repeating presolve.
Tried aggregator 3 times.
MIP Presolve eliminated 5 rows and 131 columns.
MIP Presolve modified 16 coefficients.
Aggregator did 6 substitutions.
Reduced MIP has 11 rows, 5 columns, and 42 nonzeros.
Reduced MIP has 5 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.26 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 11 rows, 5 columns, and 42 nonzeros.
Reduced MIP has 5 binaries, 0 generals, 0 SOSs, and 0 indicators.
P

Flipset with %d Items
# items: 14
x: array([1.0e+00, 2.9e+01, 9.8e+01, 0.0e+00, 1.6e+03, 0.0e+00, 9.8e+01,
       0.0e+00, 9.8e+01, 0.0e+00])
w: ([ 1.2779e-05  2.8579e-02 -4.4486e-01  3.5465e-05  1.7688e-05  7.6757e-03
 -3.2944e-01 -4.5804e-02  7.4282e-01 -8.2356e-02])
items: [{'cost': 8.460812502576125, 'feasible': True, 'status': 'integer optimal solution', 'costs': array([0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 8.4608, 0.    ,
       0.    , 0.    ]), 'actions': array([  0.,   0.,   0.,   0.,   0.,   0., -96.,   0.,   0.,   0.]), 'upperbound': 8.460812502576125, 'lowerbound': 8.460812502576125, 'gap': 0.0, 'iterations': 13, 'nodes_processed': 0, 'nodes_remaining': 0, 'runtime': 0.0}, {'cost': 8.461929932338679, 'feasible': True, 'status': 'integer optimal solution', 'costs': array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1174e-03,
       0.0000e+00, 8.4608e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]), 'actions': array([  0.,   0.,   0.,   0.,  10.,   0., -96.,   0.,  

In [25]:
from IPython.display import HTML
HTML(fb.to_html())

## Train classifiers

In [26]:
from copy import deepcopy as copy

In [28]:
from sklearn.model_selection import StratifiedKFold as CVGenerator
from sklearn.model_selection import GridSearchCV

cv_generator = CVGenerator(n_splits = 10, random_state = settings['randomseed'])

# this code is for general purpose train/test evaluation using GridSearchCV
gridsearch = GridSearchCV(
    clf, param_grid={"C":[1.0 / l for l in [1, 2]]},
                          #, 5, 10, 20, 50, 100, 200, 500, 1000]]},
    scoring='neg_mean_squared_error',
    return_train_score=True,
    cv=cv_generator,
    verbose=1,
    n_jobs=-1
)

if settings['normalize_data']:
    gridsearch.fit(X_clf_scaled, y_clf)
else:
    gridsearch.fit(X_clf_scaled, y_clf)
grid_search_df = pd.DataFrame(gridsearch.cv_results_)

Fitting 10 folds for each of 2 candidates, totalling 20 fits


/opt/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  6.3min finished


In [30]:
# cache a model for each parameter combination, trained on all data
from tqdm import tqdm_notebook

model_dict = {}
grid_search_df['key'] = pd.np.nan
for idx, p in tqdm_notebook(list(grid_search_df.params.iteritems())):

    model = copy(clf.set_params(**p)).fit(X_clf_scaled, y_clf)
        
    key = '__'.join(map(lambda x: '%s_%s' % x, p.items()))
    model_dict[key] = model
    grid_search_df.loc[idx, 'key'] = key

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [31]:
model_stats_df = refomat_gridsearch_df(
    grid_search_df,
    settings=settings,
    n_coefficients = X_clf_scaled.shape[1],
    invert_C=settings['method_name'] == 'logreg'
)

## run audit

In [33]:
audit_results = {}
for key, clf in model_dict.items():
    if settings['method_name'] == 'logreg':
        model_name = 1. / float(key.split('_')[1])
    else:
        model_name = float(key.split('_')[1])
        
    # unscale coefficients
    if scaler is not None:
        coefficients, intercept = undo_coefficient_scaling(coefficients = np.array(clf.coef_).flatten(), intercept = clf.intercept_[0], scaler = scaler)
    else:
        coefficients, intercept = np.array(clf.coef_).flatten(), clf.intercept_[0]

    ## run audit
    print("Auditing for model %s..." % key)
    auditor = RecourseAuditor(
        action_set,
        coefficients = coefficients,
        intercept = intercept
    )
    audit_results[model_name] = auditor.audit(X = X_clf_scaled)


## cache
# if settings['save_flag']:
#     pickle.dump(audit_results, file = open(settings['audit_file'], 'wb'), protocol=2)

Auditing for model C_1.0...


AssertionError: 